### Prepare Repo

In [1]:
# !git clone https://github.com/facebookresearch/av_hubert.git

%cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

%cd fairseq
# !pip install ./

/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert
/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq


In [2]:
!pip install omegaconf==2.1.1
!pip install hydra-core==1.1.1
# !pip install -U numpy==1.23.5
# !apt-get update && apt-get install -y python3-opencv
# !pip install opencv-python
# !pip install scikit-image 
# !pip install transformers
# !pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install wandb

  Obtaining dependency information for accelerate>=0.20.3 from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.4 MB/s eta 0:00:00


In [3]:
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from transformers import Trainer, TrainingArguments
import sys
sys.path.append("/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert")
%cd /home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import wandb
from torch.utils.data import Dataset, Subset
import os
import datetime

/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert
3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]


### Download Model

In [4]:
# os.makedirs("/home/check_pts/")
# !wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O /home/check_pts/avhubert.pt

### Build Model Pipeline

In [5]:
user_dir = "/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "/home/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Checkpoint: fine-tuned


### Load Dataset

In [6]:
import torch
print(torch.__version__)

2.1.0


In [7]:
from avhubert_ds import AVHUBERTDataset
mmser_ds = torch.load("/home/avhubert_ds2.pt")
mmser_ds.video_path = "/home/face_raw/"

outputs = model.extract_finetune(mmser_ds[:2])
print(outputs[0].shape)

torch.Size([2, 500, 768])


### Define the model

In [8]:
from avhubert_classifier import AVHUBERTClassifier

In [9]:
classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
classifier(**mmser_ds[:4])

{'logits': tensor([[-0.0725,  0.0250, -0.0183,  0.1193],
         [-0.0547,  0.0286, -0.0049,  0.0939],
         [-0.0926,  0.0336, -0.0364,  0.1108],
         [-0.1030,  0.0324, -0.0474,  0.0960]], grad_fn=<AddmmBackward0>)}

### Build Train Test DS

In [10]:
meta_df_ = mmser_ds.df_
sess_dict = mmser_ds.df_.groupby("session").groups
all_indices = set(mmser_ds.df_.index.tolist())

sess_ds = {}
for sess in sess_dict:
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])

In [11]:
def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

### Run Pipeline

API: 2999b8f99f0f62b4f64c48a1c8be9a16945183e9

In [12]:
import json
sess_id = list(sess_dict.keys())[7]
print("="*10, sess_id, "="*10)

avhubert_classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
for param in avhubert_classifier.encoder.parameters():
    param.requires_grad = False

wandb.init()
print(sess_id)
train_set, val_set, test_set = build_ds(sess_id)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


========== Ses04M ==========


wandb: Currently logged in as: mmser. Use `wandb login --relogin` to force relogin


Ses04M
Train Samples: 3789
Val Samples: 1264
Test Samples: 478


In [13]:
output_dir=os.path.join("check_pts", "AVHUBERT", sess_id, datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=7e-4
training_args.num_train_epochs=50

In [14]:
from avhubert_trainer import CustomTrainer, compute_metrics

trainer = CustomTrainer(
    model=avhubert_classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/multi_modal_ser/finetune_encoder/audio_video/avhubert_classifier.py", line 34, in forward
    outputs = self.encoder.extract_finetune(
  File "/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert/hubert.py", line 710, in extract_finetune
    features_video = self.forward_features(src_video, modality='video')
  File "/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert/hubert.py", line 541, in for

In [ ]:
eval_result = trainer.evaluate()
test_result = trainer.predict(test_set).metrics

In [ ]:
FREEZE_PROJ_PATH = "/home/freeze/{}/projector".format(sess_id)
FREEZE_CLAS_PATH = "/home/freeze/{}/classifier".format(sess_id)
os.makedirs(FREEZE_PROJ_PATH, exist_ok=True)
os.makedirs(FREEZE_CLAS_PATH, exist_ok=True)

FREEZE_PROJ = os.path.join(FREEZE_PROJ_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")
FREEZE_CLAS = os.path.join(FREEZE_CLAS_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")

torch.save(avhubert_classifier.projector.state_dict(), FREEZE_PROJ)
torch.save(avhubert_classifier.classifier.state_dict(), FREEZE_CLAS)

model.projector.load_state_dict(torch.load(FREEZE_PROJ))
model.classifier.load_state_dict(torch.load(FREEZE_CLAS))

print(eval_result)
print(test_result)

json_test = json.dumps(test_result, indent=4)
json_eval = json.dumps(eval_result, indent=4)

# Writing to sample.json
with open("{}_eval.json".format(sess_id), "w") as outfile:
    outfile.write(json_eval)
with open("{}_test.json".format(sess_id), "w") as outfile:
    outfile.write(json_test)


In [ ]:
meta_df_ = mmser_ds.df_
sess_dict = meta_df_.groupby("session").groups
all_indices = set(meta_df_.index.tolist())
sess_ds = {}
for sess in sess_dict:
#     sess_ds[sess+"_train"] = Subset(mmser_ds, 
#                                     indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    
    
    
SESS_ID = list(sess_dict.keys())[0]
train_set, val_set, test_set = build_ds(SESS_ID)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=2)
# next(iter(train_loader))

In [ ]:
output_dir=os.path.join("avhubert")

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = 3 # int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = 3 #int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=1e-3
training_args.num_train_epochs=50

trainer = CustomTrainer(
    model=classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()